<a href="https://colab.research.google.com/github/sizhky/naive-bayes-demo/blob/main/spam-ham/step_4_model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install torch_snippets

At this point we have the best model, vectorizer and model-paramter for our task. Final step is to create a class that can be used in production. Typically, in production scenario, we want to expose a one line function that takes inputs and returns outputs in readable format. Most of the time, this one line function will be used to expose an API, but we will not do that. We will only have code necessary to train and create a predict function

In [2]:
## Setup and imports
import os
from pathlib import Path
import urllib.request
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_snippets import unzip_file

if not os.path.exists('SMSSpamCollection'):
    urllib.request.urlretrieve ("https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip", "smsspamcollection.zip")
    # Extracting the dataset
    unzip_file('smsspamcollection.zip', './')

2021-07-29 05:54:57.502 | WARNING  | torch_snippets.torch_loader:<module>:233 - Not importing Lightning Report
2021-07-29 05:54:57.507 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

class SpamDetector:
    classes = ["HAM", "SPAM"]
    vectorizer = CountVectorizer(
        stop_words='english'
    )

    df = pd.read_csv('SMSSpamCollection', sep='\t', header=None)
    df.columns = ['class','content']
    trn_df, val_df = train_test_split(df)

    vectorizer.fit(trn_df['content'])
    trn = vectorizer.transform(trn_df['content']).todense()
    val = vectorizer.transform(val_df['content']).todense()

    nb = MultinomialNB(alpha=0.3)
    nb.fit(trn, trn_df['class'])

    def predict(self, sentence):
        if isinstance(sentence, str):
            sentence = [sentence]
        test = self.vectorizer.transform(sentence).todense()
        print(sentence)
        df = pd.DataFrame({
            'input': sentence,
            'prediction': self.nb.predict(test), 
            'confidence': self.nb.predict_proba(test).max(-1)
        })
        return df

detector = SpamDetector()

In [4]:
detector.predict(['Happy new year', 'Free mobile offer, hurry up'])

['Happy new year', 'Free mobile offer, hurry up']


,input,prediction,confidence
0,Happy new year,ham,0.994726
1,"Free mobile offer, hurry up",spam,0.992890
